# The Ramsey notebook  <a id="Ramsey"></a>[<font size=1>(back to `Main.ipynb`)</font>](./Main.ipynb)

This notebook gathers the computations of the steady-state Ramsey solution for the truncated model. 

Thz computation mostly follows the matrix computation of the [paper](https://francois-le-grand.fr/docs/research/LRR_Welfare-of-nations.pdf).

To sum it up, we aim at simulating the Ramsey model using perturbation methods via Dynare. To do so, we need to determine the steady-state Ramsey allocation. Our computational strategy consists in finding the Pareto weights of the SWF such that the observed US tax system is optimal at the steady state. It means that the truncated allocation `truncatedModel::TruncatedModel` computed by the function `TruncatedModel` of the notebook [Truncation.ipynb](./Truncation.ipynb) yields the Ramsey allocation (because it has been computed for the observed US fiscal system). However, Ramsey FOCs depend on the Ramsey Lagrange multipliers as well as the Pareto weights and the marginal value of liquidity $(\psi_{h})_h$. We thus need to compute the steady-state values of these quantities.

We do this in two steps. [First](#Ramsey-LM), we express Lagrange multipliers and marginal value of liquidity as a function of Pareto weights only using the truncated allocation `truncatedModel::TruncatedModel`. [Second](#Ramsey-weights), we compute the Pareto weights that respect the Ramsey FOCs, such that the SWF is the closest to the utilitarian welfare. 

Finally, we present the [implementation](#Ramsey-implementation).

## The Lagrange multipliers <a id="Ramsey-LM"></a>[<font size=1>(back to Ramsey)</font>](#Ramsey)

Computing the Lagrange multipliers as a function of the Pareto weights using the truncated allocation is not difficult but involves some tedious algebra. However, we take advantage of the finite state space that makes it possible to express these Lagrange multipliers (e.g., $(\lambda_{c,h})_h$ for the Euler equations) as vectors.  The resulting implementation is fast as it mostly involves basic linear algebra. The matrix representation consists in stacking together the equations characterizing the steady state, so as to provide a convenient matrix notation for solving the steady state.


### Preliminary definitions 

We start with a number of definitions that will allow us to express Lagrange multipliers as function of Pareto weight using the truncated allocation: `truncatedModel::TruncatedModel` of the notebook [Truncation.ipynb](./Truncation.ipynb).

We denote with a bold letter the vector associated to a given variable. For instance $\boldsymbol{c} := (c_h)_{h\in\mathcal H}$ corresponds to the vector of individual consumption levels in truncated histories $h$. The order in which the histories are stacked does not matter but the order has to be consistent from one vector (or matrix) to another. 

We now define a number of quantities. 

* We define the matrix: $\bar{\boldsymbol{\Pi}} = \boldsymbol{S} \circ \boldsymbol{\Pi}^\top \circ (1/\boldsymbol{S})$, where $\boldsymbol{S}$ is the size (as a vector) of histories (variable `S_h`) and $\Pi^\top$ the transpose of the transition matrix for histories (variable `Π_h` -- be careful that this matrix is actually the transpose of $\boldsymbol{\Pi}$).
* We define the diagonal matrix $\boldsymbol{P}$ (`P`) with $1$ on the diagonal if history $h$ is not constrained and $0$ otherwise. 
* The matrix $\boldsymbol{D}_\boldsymbol{x}$ is the diagonal matrix with $\boldsymbol{x}$ on the diagonal. Note that $\boldsymbol{D}_\boldsymbol{x}^{-1} = D_{1/\boldsymbol{x}}$ if no element of $\boldsymbol{x}$ is null.


Using this notation, we define a series of matrices and vectors:

* First:
\begin{align*} 
\boldsymbol{M}_{1}&:=\boldsymbol{D}_{\tilde{\boldsymbol{\xi}}^{v,1}\circ v^{\prime\prime}(\boldsymbol{l})+\tau\tilde{\boldsymbol{\boldsymbol{\xi}}}^{u,1}\circ u^{\prime}(\boldsymbol{c})}^{-1},\\
\boldsymbol{M}_{0}&:=-\boldsymbol{M}_{1}\boldsymbol{D}_{\tilde{\boldsymbol{\boldsymbol{\xi}}}^{v,0}\circ v^{\prime}(\boldsymbol{l})}, \\
\boldsymbol{V}_{0}&:=F_{L}\boldsymbol{M}_{1}\boldsymbol{S}./((1-\tau)W\boldsymbol{\xi}^{y}\circ\boldsymbol{y}^{-\tau}\circ\boldsymbol{l}^{-\tau}),
\end{align*} 

which are represented by the matrices `M1` and `M0` and the vector `V0`, respectively.

* Second: 
\begin{align*} 
\hat{\boldsymbol{M}}_{0}&:=\boldsymbol{D}_{\boldsymbol{\xi}^{u,0}\circ u^{\prime}(\boldsymbol{c})},\\\hat{\boldsymbol{M}}_{1}&:=-\boldsymbol{D}_{\boldsymbol{\xi}^{u,E}\circ u^{\prime\prime}(\boldsymbol{c})}(\boldsymbol{I}-R\boldsymbol{\Pi}),\\\hat{\boldsymbol{M}}_{2}&:=(1-\tau)W\boldsymbol{D}_{\boldsymbol{\xi}^{y}\circ(\boldsymbol{y}\circ\boldsymbol{l})^{1-\tau}\circ\tilde{\boldsymbol{\xi}}^{u,1}\circ u^{\prime\prime}(\boldsymbol{c})},
\end{align*} 

which are represented by the matrices `M0h`, `M1h`, and `M2h` (do not forget that $\boldsymbol{\Pi}$ corresponds to the transpose of `Π_h`).
* Third:
\begin{align*}
\boldsymbol{M}_{2}	&:=\boldsymbol{I}-\hat{\boldsymbol{M}}_{2}\boldsymbol{M}_{1},\\
\boldsymbol{M}_{3}	&:=\boldsymbol{M}_{2}^{-1}(\hat{\boldsymbol{M}}_{0}+\hat{\boldsymbol{M}}_{2}\boldsymbol{M}_{0}),\\
\boldsymbol{M}_{4}	&:=\boldsymbol{M}_{2}^{-1}\hat{\boldsymbol{M}}_{1},\\
\boldsymbol{V}_{1}	&:=\boldsymbol{M}_{2}^{-1}(\hat{\boldsymbol{M}}_{2}\boldsymbol{V}_{0}-\boldsymbol{S}),
\end{align*}

which are represented by the matrices `M2`, `M3`, and `M4` and the vector `V1`. Note that we use the left division (with `\`) rather than the computation of the inverse. Indeed, `A\B` is more stable and faster than `inv(A)*B` -- but both yield the same result when `A` is invertible

* Fourth: 
\begin{align*}
\tilde{\boldsymbol{R}}_{5}	&:=-((\boldsymbol{I}-\boldsymbol{P})+\boldsymbol{P}(I-\beta R\bar{\boldsymbol{\Pi}})\boldsymbol{M}_{4})^{-1}\boldsymbol{P}(I-\beta R\bar{\boldsymbol{\Pi}}),\\
\boldsymbol{M}_{5}	&:=\tilde{\boldsymbol{R}}_{5}\boldsymbol{M}_{3},\\
\boldsymbol{V}_{2}	&:=\tilde{\boldsymbol{R}}_{5}\boldsymbol{V}_{1},
\end{align*}

which are represented by the matrices `R5t` and `M5` and the vector `V2`.

* Fifth:
\begin{align*}
\boldsymbol{C}_{1}	&:=\boldsymbol{\tilde{a}}^{\top}(\boldsymbol{V}_{1}+\boldsymbol{M}_{4}\boldsymbol{V}_{2})+(\boldsymbol{\xi}^{u,E}\circ u^{\prime}(\boldsymbol{c}))^{\top}\boldsymbol{\Pi}\boldsymbol{V}_{2},\\
\boldsymbol{L}_{1}	&:=\left(\boldsymbol{\tilde{a}}^{\top}(\boldsymbol{M}_{3}+\boldsymbol{M}_{4}\boldsymbol{M}_{5})+(\boldsymbol{\xi}^{u,E}\circ u^{\prime}(\boldsymbol{c}))^{\top}\boldsymbol{\Pi}\boldsymbol{M}_{5}\right)/\boldsymbol{C}_{1},
\end{align*}

which are represented by the scalar `C1` and the vector `L1`. Note that $\boldsymbol{\tilde{a}}$ corresponds to variable `a_beg_h`.

* Finally:
\begin{align*}
\boldsymbol{M}_{6} &:= \boldsymbol{M}_{3}+\boldsymbol{M}_{4}(\boldsymbol{M}_{5}-\boldsymbol{V}_{2}\boldsymbol{L}_{1})-\boldsymbol{V}_{1}\boldsymbol{L}_{1}, \\
\hat{\boldsymbol{M}}_{6}&:=\boldsymbol{M}_{0}+\boldsymbol{M}_{1}\boldsymbol{M}_{6}-\boldsymbol{V}_{0}\boldsymbol{L}_{1},\\
\boldsymbol{L}_{2}&:=\left(\ln(\boldsymbol{y}\circ\boldsymbol{l})\circ\boldsymbol{\xi}^{y}\circ(\boldsymbol{y}\circ\boldsymbol{l})^{1-\tau}\right)^{\top}\boldsymbol{M}_{6} \\
&+\left((\boldsymbol{1}+(1-\tau)\ln(\boldsymbol{y}\circ\boldsymbol{l}))\circ\boldsymbol{\xi}^{y}\circ(\boldsymbol{y}\circ\boldsymbol{l})^{1-\tau}\circ\tilde{\boldsymbol{\mathbf{\xi}}}^{u,1}\circ u^{\prime}(\boldsymbol{c})\right)^{\top}\hat{\boldsymbol{M}}_{6},\\
\boldsymbol{L}_{3} &:= \left(\boldsymbol{\xi}^{y}\circ(\boldsymbol{y}\circ\boldsymbol{l})^{1-\tau}\right)^{\top}\boldsymbol{M}_{6} \\ 
&+ (1-\tau)\left(\boldsymbol{\xi}^{y}\circ(\boldsymbol{y}\circ\boldsymbol{l})^{1-\tau}\circ\tilde{\boldsymbol{\mathbf{\xi}}}^{u,1}\circ u^{\prime}(\boldsymbol{c})\right)^{\top}\hat{\boldsymbol{M}}_{6},
\end{align*}

which are represented by the matrices `M6` and `M6h` and the vectors `L2` and `L3`.

### Lagrange multipliers

Using the above definitions, we can compute the various quantities, which are:
* the Pareto weights of the SWF: vector $\boldsymbol{\omega}$ and variable `ω`,
* the marginal value of public fund $\psi$: vector $\boldsymbol{\psi}$ and variable `ψ`,
* the Lagrange multiplier for the Euler equation $(\lambda_{c,h})_h$: vector $\boldsymbol{\lambda}_{c}$ and variable `λc`,
* the one for the labor FOC $(\lambda_{l,h})_h$ (vector `λl`), 
* the one for the governmental budget constraint $\mu$ (scalar `μ`),

as well as the associated variables (quantities at the history level instead of the individual one): $\bar{\boldsymbol{\omega}} :=\boldsymbol{S} \circ \boldsymbol{\omega}$ (variable `ωb`), $\bar{\boldsymbol{\psi}} :=\boldsymbol{S} \circ \boldsymbol{\psi}$ (variable `ψb`), $\bar{\boldsymbol{\lambda}}_{c} :=\boldsymbol{S}\circ\boldsymbol{\lambda}_{c}$ (variable `λcb`),  $\bar{\boldsymbol{\lambda}}_{l} :=\boldsymbol{S}\circ\boldsymbol{\lambda}_{l}$ (variable `λlb`). Additionally, there is the average past Lagrange multiplier for the Euler equation: $(\tilde\lambda_{c,h})_h$: vector $\tilde{\boldsymbol{\lambda}}_{c}$ and variable `λct`.

We have the following relationships:
\begin{align*}
    \tilde{\boldsymbol{\lambda}}_{c} &= \boldsymbol{\Pi}\boldsymbol{\bar{\lambda}}_{c},
    \mu=-\boldsymbol{L}_{1}\boldsymbol{\bar{\omega}},\\
    \boldsymbol{\bar{\lambda}}_{c} &=(\boldsymbol{M}_{5}-\boldsymbol{V}_{2}\boldsymbol{L}_{1})\boldsymbol{\bar{\omega}},\\
    \boldsymbol{\bar{\psi}} &= \boldsymbol{M}_{6}\boldsymbol{\bar{\omega}},\\
    \boldsymbol{\bar{\lambda}}_{l} &= \hat{\boldsymbol{M}}_{6}\boldsymbol{\bar{\omega}}.
\end{align*}

which are solely expressed as a function of the Pareto weights $\boldsymbol{\bar{\omega}}$.

## The weights determination  <a id="Ramsey-weights"></a>[<font size=1>(back to Ramsey)</font>](#Ramsey)


The Ramsey  FOCs deliver two additional constraints: 
\begin{align*}
    \boldsymbol{L}_{2}\boldsymbol{\bar{\omega}} &= 0, \\
    \boldsymbol{L}_{3}\boldsymbol{\bar{\omega}} &= 0.
\end{align*}

Because of the dimensionality of $\boldsymbol{\bar{\omega}}$ (a vector of length $2^{|\mathcal Y|}\ge 4$, where $|\mathcal Y|$ is the number of idiosyncratic risk), these two constraints do not allow one to pin down $\boldsymbol{\bar{\omega}}$. 

We start with reducing the dimensionality of $\boldsymbol{\bar{\omega}}$ and assuming that the Pareto weights does not depend on the whole truncated history but on the current productivity level only, as in [Heathcote and Tsujiyama (2021)](https://www.journals.uchicago.edu/doi/abs/10.1086/715851). We denote these productivity-dependent Pareto weights by the vector $ \boldsymbol{\omega}^{s}$ of length $|\mathcal Y|$. Defining by $ \boldsymbol{M}_{7}$ as the $N_{tot}\times|\mathcal{Y}|$ matrix, whose element $m_{hy}$ is 1 if history $h$ has current productivity $y$, we have: 
\begin{align*}
    \bar{\boldsymbol{\omega}}=\boldsymbol{D}_{\boldsymbol{S}}\boldsymbol{M}_{7}\boldsymbol{\omega}^{s}. 
\end{align*}

However, $|\mathcal{Y}|$ is typically greater than 5 in applications and in spite of the previous dimensionality reduction, the vector $ \boldsymbol{\omega}^{s}$ remains underidentified. To solve this issue, we seek the minimize that respect previous constraints and are the closest to constant weighs (corresponding to a utilitarian planner). Formally, if we denote the constant value by $\underline{\omega}$, the weights $\boldsymbol{\omega}^{s}$ can be written as the solution of the following program:
\begin{align*}
    \min_{\boldsymbol{\omega}^{s}\in \mathbb R_+^{|\mathcal Y|}}& \sum_{y\in\mathcal Y} S_y(\omega_y^s -\underline{\omega})^2, \\ 
    \text{s.t.}\quad & \boldsymbol{L}_{2}\boldsymbol{D}_{\boldsymbol{S}}\boldsymbol{M}_{7}\boldsymbol{\omega}^{s} = 0, \\
    &\boldsymbol{L}_{3}\boldsymbol{D}_{\boldsymbol{S}}\boldsymbol{M}_{7}\boldsymbol{\omega}^{s} = 0.
\end{align*} 
    
We restrict to positive weights for economic reasons: otherwise, household could be "sacrified" to increase aggregate welfare. Furthermore, note that if all weights of $\boldsymbol{\omega}^{s}$ are constant and equal to $\underline{\omega}$, then their weighted mean is also $\underline{\omega}$ since $\sum_{y\in\mathcal Y} S_y=1$. In other words, our criterion consists in minimizing the variance of $(\omega^s_y)_y$ (computed using the $(S_y)_y$). Given our program, the solution is determined up to a positive scalar (formally if $(\omega_y)_y$ is a solution then $(\lambda\omega_y)_y$ for any $\lambda>0$ is also a solution). Without loss of generality, we can add another constraint, which we choose to set the sum of the weights equal to 1: $\sum_y S_y \omega_y = 1$. 

The final program is thus: 
\begin{align*}
    \min_{\boldsymbol{\omega}^{s}\in \mathbb R_+^{|\mathcal Y|}}& \sum_{y\in\mathcal Y} S_y(\omega_y^s -\underline{\omega})^2, \\ 
    \text{s.t.}\quad & \boldsymbol{L}_{2}\boldsymbol{D}_{\boldsymbol{S}}\boldsymbol{M}_{7}\boldsymbol{\omega}^{s} = 0, \\
    &\boldsymbol{L}_{3}\boldsymbol{D}_{\boldsymbol{S}}\boldsymbol{M}_{7}\boldsymbol{\omega}^{s} = 0 ,\\
    &\sum_{y\in\mathcal Y} S_y \omega_y^s=1,
\end{align*} 
that will be solved using the package `JuMP`.





## The implementation  <a id="Ramsey-implementation"></a>[<font size=1>(back to Ramsey)</font>](#Ramsey)

The computation of the Ramsey allocation `ramsey_Solution::Ramsey` can be performed as follows:

> `ramsey_Solution = Ramsey(truncatedModel::TruncatedModel,solution::AiyagariSolution,economy::Economy)`

through a specific constructor of the struct `Ramsey`. 

This function takes as inputs: 
* economy parameters `economy::Economy`,
* the solution of the Aiyagari model `solution::AiyagariSolution` corresponding to `economy`, 
* the truncated allocation `truncatedModel::TruncatedModel` of `solution`.

The struct `ramsey_Solution` contains the following elements.
* `weights::Weights` containing the SWF weights, and more precisely the vector of weights  $\boldsymbol{\omega}$ with length $|\mathcal Y|$ (variable `ω`), and their transformation $\overline{\boldsymbol{\omega}}$ (variable `ωb`). 
* `lagrangeMult::LagrangeMult` containing the steady-state values of the Lagrange multipliers of the Ramsey program used for  the Dynare simulation. 
* The truncated model `truncatedModel::TruncatedModel`.


In [1]:
function Ramsey(truncatedModel::TruncatedModel,solution::AiyagariSolution,economy::Economy)#::Ramsey

    @unpack N,Ntot,ind_h,truncatedAllocation,ξs = truncatedModel
    @unpack S_h,Π_h,y0_h,a_beg_h,a_end_h,c_h,l_h,u_h,u′_h,u′′_h,v_h,v′_h,nb_cc_h,ind_cc_h = (
                truncatedAllocation)
    @unpack ξu0,ξu1,ξuE = ξs 
    @unpack β,α,δ,u′,u′′,v′,v′′,ys,ny,Sy,ψw,ϵw,κw,τk = economy
    @unpack w,R,K,L,G,Y,A, B = solution

    T = typeof(w)
    # FL = (1-α)*K^α*L^-α
    
    r = R - 1
    rt = r / (1-τk) 

    
    ####
    ## Preliminary definitions
    ####
    
    # Defining some specific matrices
    Π_h_bar   = spdiagm(S_h)*Π_h*spdiagm(one(T)./S_h) #### verify ∀x :  Π_h_ψ*(S_h.*x)=S_h.(Π_h*x)  
    Is        = sparse(I,Ntot,Ntot)
    P         = Is - sparse(ind_cc_h,ind_cc_h,ones(T,nb_cc_h),Ntot,Ntot)   
    
    ####
    ## Defining the Lagrange multipliers as a function of SWF weights
    ####
    
    # We define a number of matrices and vectors in the same order as in the presentation above 
    # (separated in blocks)
    
    # First block
    L1t = Is - β*(1+r)*Π_h_bar + (β/K)*(1-α)*(r+(1-τk)*δ)*S_h*a_beg_h'- (β/K)*α*w*L*S_h*(y0_h)' 
    M1  =  (β/K)*(α-1)*(r+ (1-τk)*δ)*S_h*(ξuE.*u′.(c_h))'*Π_h' 
    L1  =  (β/K)*(α*Y - (α-1)*(r+ (1-τk)*δ)*A - (r+δ)*K - α*w*L)*S_h
    N1  =  - β*α*((ϵw-1)/ψw)*w*L/K*S_h*(y0_h)'*(S_h.*u′.(c_h))

    # Second block
    M2  = - spdiagm(ξuE.*u′′.(c_h))*(Is-(1+r)*Π_h') 
    L2  = S_h.*ξu0.*u′.(c_h)
    L2t = - ( (ϵw-1)/ψw )*w*L*(S_h.*y0_h.*ξu1.*u′′.(c_h))

    #Third block
    #  x = inv(M)b \equiv x = M\b
    M3  = I - P + P*(L1t*M2 - M1) 
    L3  = - (M3)\(P*L1t*L2)
    L3t = (M3)\(P*L1)
    L3h =  (M3)\(P*(N1-L1t*L2t))

    # Fourth block
    L4  = M2*L3 + L2
    L4t = M2*L3t
    L4h = M2*L3h + L2t
        
    # Fifth block
    C5t = (1-α)*(1/L)*(Y - A*(rt+ δ)*(1-τk) - w*L)
    C5h =  (ϵw/ψw)*( v′(L) + v′′(L)*L - (1-α)*((ϵw-1)/ϵw)*w*y0_h'*(S_h.*u′.(c_h)) )
    L5  = (1-α)*w*y0_h' + (1-α)*(1-τk)*(rt+δ)/L*a_beg_h'
    L5t =   (1-α)*(1-τk)*(rt+δ)/L*(ξuE.*u′.(c_h))'*Π_h' 

    # Sixth block
    C6  = L5*L4 + L5t*L3 -  v′(L) 
    C6t = L5*L4t + L5t*L3t + C5t
    C6h = L5*L4h + L5t*L3h + C5h
        
    # Seventh block
    C7 = ones(T,1,Ntot)*L4
    C7t = ones(T,1,Ntot)*L4t .- 1
    C7h = ones(T,1,Ntot)*L4h 
        
    # Defining the matrices M7 and DS    
    ind_ys = Dict(value => key for (key, value) in enumerate(ys))#indices as a function of 
                                                                 #productivity level
    ind_y0_h  = [ind_ys[y0] for y0 in y0_h] # productivity indices of 
    M7        = sparse(1:Ntot,ind_y0_h,1.0)
    DS        = spdiagm(S_h)
    
    # check that DS and M7 are well-defined
    @assert maximum(abs.((ones(T,1,length(DS[1,:]))*DS*M7)[:] .- Sy)) < 1e-10
   
    # Lagrange multipliers    
    # Λ  = 0 
    γw  = - (C7t*C6 - C6t*C7)/(C7t*C6h - C6t*C7h)
    μ  =  - (C7h*C6 - C6h*C7)/(C6t*C7h - C7t*C6h)

    ψb = L4 + L4t*μ + L4h*γw 
    ψ   = (1.0 ./ S_h).*ψb

    λcb = L3 + L3t*μ + L3h*γw
    λc  = (1.0 ./ S_h).*λcb
    λct = (1.0 ./ S_h).*(Π_h' * λcb)

    Γ = μ - (1/A)*(a_beg_h'*ψb + (ξuE.*u′.(c_h))'*Π_h' * λcb) 

    υ  =  β*(1-τk)*Γ*B 




    lagrangeMult = LagrangeMult(
        λc  = vec((1.0 ./ S_h).*λcb),
        λct = vec((1.0 ./ S_h).*(Π_h' * λcb)),
        λcb = vec(λcb),
        μ   = μ[1],
        Λ   = zero(T),
        γw  = γw[1],
        υ  = υ[1],
        Γ  = Γ[1],
        ψ   = vec(ψ),
        ψb  = vec(ψb))
    


return Ramsey(lagrangeMult=lagrangeMult, truncatedModel=truncatedModel)

end;

LoadError: LoadError: UndefVarError: `@unpack` not defined
in expression starting at /Users/am13232/Dropbox/optimal_money_New/Code_Wage/AMB/Sticky_Wages/steady-state/Ramsey_SW.ipynb:3

### Checking the solution

We check below different aspects of the Ramsey solution.
* we check that bar et non-bar variables are consistent (e.g., $\bar{\boldsymbol{\omega}} = \boldsymbol{S} \circ \boldsymbol{\omega}$);
* we check that $\boldsymbol{S} \circ \tilde{\boldsymbol{\lambda}}_c = \Pi\,\bar{\boldsymbol{\lambda}}_c$; 
* we check that the definitions of $\tilde{\boldsymbol{\xi}}^{v,i}$ and ${\boldsymbol{\xi}}^{v,1}$  are consistent;
* we check that $\boldsymbol{S}$ is a eigenvector $\boldsymbol{\Pi}$ with eigenvalue 1;
* we check that the definitions of $\tilde{\boldsymbol{a}}$ and $\boldsymbol{a}$ are consistent with each other;
* we check that the budget constraint holds at the history level;
* we chack that the FOCs hold (wrt $a_h$, $l_h$, $R$, $w$, $\tau$ in this order);
* we check that $\psi_h$ is null if $h$ is credit-constrained;
* we check that the definition of $\bar{\boldsymbol{\psi}}$ holds.

In [2]:
# function check_ramsey(ramsey::Ramsey,solution::AiyagariSolution,economy::Economy; 
#         opti_T::Bool=true, noprint::Bool=false)::Nothing

#     @unpack weights,lagrangeMult,truncatedModel = ramsey
#     @unpack N,Ntot,ind_h,truncatedAllocation,ξs = truncatedModel
#     @unpack S_h,Π_h,y0_h,a_beg_h,a_end_h,c_h,l_h,u_h,u′_h,u′′_h,v_h,v′_h,nb_cc_h,ind_cc_h = (
#                 truncatedAllocation)
#     @unpack ξu0,ξu1,ξuE = ξs 
#     @unpack β,α,δ,u′,u′′,v′,v′′,inv_v′,ys,ny,na,Tt,ϵw,ψw,κw = economy
#     @unpack w,R,K,L,G,Y,A,stationaryDist,gc = solution
#     @unpack λcb,ψb,λlb,λc,λct,λl,λlt,ψ,μ,γw = lagrangeMult
#     @unpack ωb_h,ω_h = weights
    
#     ψhb = ψb .- μ*S_h 
    
    
#     #@show v′(L) - κw*(ϵw-1)/ϵw*w*sum(y0_h.*S_h.*ξu1.*u′.(c_h))
    
#     #=ξu1_tilde = ξu1./l_h
#     ξv0_tilde = ξv0./((1-τ)*w*ly_τ_h./l_h)
#     ξv1_tilde = ξv1./((1-τ)*w*ly_τ_h./l_h)=#
#     FL = (1-α)*K^α*L^-α
#     Is        = sparse(I,Ntot,Ntot)
#     P         = Is - sparse(ind_cc_h,ind_cc_h,ones(typeof(β),nb_cc_h),Ntot,Ntot)   
#     Π_h_bar   = spdiagm(S_h)*Π_h*spdiagm(one(typeof(β))./S_h) #### verify ∀x :  Π_h_ψ*(S_h.*x)=S_h.(Π_h*x)  
        
#     norm(x) = maximum(abs.(x))
    
#     diffs    = ωb_h - S_h.*ω_h
#     if !(norm(diffs) < 1e-10)
#         @warn("error in ω.  Diff (ωb - S_h.*ω):   ", round.(diffs,digits=4))
#     else
#         (!noprint) && println("Passed: ω.  Diff ||ωb - S_h.*ω||∞:   ", round.(norm(diffs),digits=4))
#     end
#     diffs    = λcb - S_h.*λc
#     if !(norm(diffs) < 1e-10)
#         @warn("error in λc. Diff (λcb - S_h.*λc): ", round.(diffs,digits=4))
#     else
#         (!noprint) && println("Passed: λc. Diff ||λcb - S_h.*λc||∞: ", round.(norm(diffs),digits=4))
#     end
#     diffs    = λlb - S_h.*λl
#     if !(norm(diffs) < 1e-10)
#         @warn("error in λl. Diff (λlb - S_h.*λl): ", round.(diffs,digits=4))
#     else
#         (!noprint) && println("Passed: λl. Diff ||λlb - S_h.*λl||∞: ", round.(norm(diffs),digits=4))
#     end
#     diffs    = S_h.*λct - Π_h'*λcb
#     if !(norm(diffs) < 1e-10)
#         @warn("error in λct. Diff (S_h.*λct - Π_h'*λcb): ", round.(diffs,digits=4))
#     else
#         (!noprint) && println("Passed: λct. Diff ||S_h.*λct - Π_h'*λcb||∞: ", round.(norm(diffs),digits=4))
#     end
#     #=diffs    = ξv1 - (1-τ)*w*ξv1_tilde.*ly_τ_h./l_h
#     if !(norm(diffs) < 1e-10)
#         @warn("error in ξv1. Diff (ξv1): ", round.(diffs,digits=4))
#     else
#         println("Passed: ξv1. Diff ||(ξv1)||∞: ", round.(norm(diffs),digits=4))
#     end
#     diffs    = ξv0 - (1-τ)*w*ξv0_tilde.*ly_τ_h./l_h
#     if !(norm(diffs) < 1e-10)
#         @warn("error in ξv0. Diff (ξv0): ", round.(diffs,digits=4))
#     else
#         println("Passed: ξv0. Diff ||(ξv0)||∞: ", round.(norm(diffs),digits=4))
# end=#
#     diffs    = S_h - Π_h'*S_h
#     if !(norm(diffs) < 1e-10)
#         @warn("error in S_h. Diff (S_h - Π_h'*S_hb): ", round.(diffs,digits=4))
#     else
#         (!noprint) && println("Passed: S_h. Diff ||S_h - Π_h'*S_h||∞: ", round.(norm(diffs),digits=4))
#     end
#     diffs    = S_h.*a_beg_h - Π_h'*(S_h.*a_end_h)
#     if !(norm(diffs) < 1e-10)
#         @warn("error in a_beg_h. Diff (a_beg_h): ", round.(diffs,digits=4))
#     else
#         (!noprint) && println("Passed: a_beg_h. Diff ||a_beg_h||∞: ", round.(norm(diffs),digits=4))
#     end
#     diffs    = c_h + a_end_h - R*a_beg_h - w*L*y0_h .- Tt
#     #@show c_h[2], a_end_h[2],a_beg_h[2],  l_h[2],y0_h[2],ξy[2]
#     if !(norm(diffs) < 1e-10)
#         @show diffs
#         @warn("error in budget const. Diff (budget const): ", round.(diffs,digits=4))
#     else
#         (!noprint) && println("Passed: budget const. Diff ||budget const||∞: ", round.(norm(diffs),digits=4))
#     end
#     diffs    = P*ψhb - β*R*P*Π_h_bar*ψhb
#     if !(norm(diffs) < 1e-10)
#         @show diffs
#         @warn("error in FOC (a_h). Diff (P*ψb - β*R*P*Π_h_bar*ψb): ", round.(diffs,digits=4))
#     else
#         (!noprint) && println("Passed: FOC (a_h). Diff ||P*ψb - β*R*P*Π_h_bar*ψb||∞: ", round.(norm(diffs),digits=4))
#     end
#     diffs    = sum(ωb_h)*v′(L) - μ - (ϵw/ψw)*γw*(v′′(L)*L+v′(L))
#     if !(norm(diffs) < 1e-10)
#         @warn("error in FOC (l_h). Diff (FOC): ", round.(diffs,digits=4))
#     else
#         (!noprint) && println("Passed: FOC (l_h). Diff ||FOC||∞: ", round.(norm(diffs),digits=4))
#     end
#     diffs    = a_beg_h'*ψhb + (ξuE.*u′.(c_h))'*Π_h'*λcb
#     if !(norm(diffs) < 1e-12)
#         @warn("error in FOC (R). Diff (FOC): ", round.(diffs,digits=4))
#     else
#         (!noprint) && println("Passed: FOC (R). Diff ||FOC||∞: ", round.(norm(diffs),digits=4))
#     end
#     diffs    = y0_h'*ψhb - γw*ϵw/(ψw*w)*v′(L)
#     if !(norm(diffs) < 1e-10)
#         @warn("error in FOC (w). Diff (FOC): ", round.(diffs,digits=4))
#     else
#         (!noprint) && println("Passed: FOC (w). Diff ||FOC||∞: ", round.(norm(diffs),digits=4))
#     end
#     diffs    = y0_h'*ψhb - γw*(ϵw-1)/(ψw)*κw*sum(y0_h.*S_h.*ξu1.*u′.(c_h))
#     if !(norm(diffs) < 1e-9)
#         @warn("error in FOC (w). Diff (FOC): ", round.(diffs,digits=4))
#     else
#         (!noprint) && println("Passed: FOC (w). Diff ||FOC||∞: ", round.(norm(diffs),digits=4))
#     end
#     diffs    = v′(L) - κw*(ϵw-1)/ϵw*w*sum(y0_h.*S_h.*ξu1.*u′.(c_h))
#     if !(abs(diffs - (v′(L) - κw*(ϵw-1)/ϵw*w*sum(stationaryDist.*((repeat(ys,1,na)'.*u′.(gc)))))) < 1e-12)
#         @show diffs
#         @show v′(L) - κw*(ϵw-1)/ϵw*w*sum(stationaryDist.*((repeat(ys,1,na)'.*u′.(gc))))
#         @show diffs - (v′(L) - κw*(ϵw-1)/ϵw*w*sum(stationaryDist.*((repeat(ys,1,na)'.*u′.(gc)))))
#         @warn("error in FOC (v′(L)). Diff (FOC): ", round.(diffs,digits=4))
#     else
#         (!noprint) && println("Passed: FOC (v′(L)). Diff ||FOC||∞: ", round.(norm(diffs),digits=4))
#     end
#     diffs    = (Is-P)*λcb
#     if !(norm(diffs) < 1e-10)
#         @warn("error in CC (λbc). Diff (CC): ", round.(diffs,digits=4))
#     else
#         (!noprint) && println("Passed: CC (λbc). Diff ||CC||∞: ", round.(norm(diffs),digits=4))
#     end
#     diffs = -ψhb + ωb_h.*ξu0.*u′.(c_h) - μ*S_h - (
#         λcb.*ξuE - R*(S_h.*λct).*ξuE).*u′′.(c_h) - γw*κw*(ϵw-1)/ψw*w*L*(y0_h.*S_h.*ξu1.*u′′.(c_h))
#     if !(norm(diffs) < 1e-10)
#         @warn("error in ψb. Diff (ψb - def_ψ): ", round.(diffs,digits=4))
#     else
#         (!noprint) && println("Passed: ψb. Diff (ψb - def_ψ): ", round.(norm(diffs),digits=4))
#     end
    
#     if opti_T 
#         diffs = sum(ψhb)
#         if !(norm(diffs) < 1e-10)
#             @warn("error in ψb. Diff (∑_h ψhb)): ", round.(diffs,digits=4))
#         else
#             (!noprint) && println("Passed: ψb. Diff (∑_h ψhb): ", round.(norm(diffs),digits=4))
#         end
#     end
    
#     return nothing
# end;